In [1]:
# Installing dependencies for PDF processing and Groq API
!pip install pymupdf groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.3 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# Setting the key for the session
os.environ["GROQ_API_KEY"] = "please give yout groq api key here"
print("Environment variable set!")

Environment variable set!


Experiment: Handling Unstructured Hospital Data

I wanted to see if the model could tell the difference between "Doctor Notes" and "Medical History" without getting them mixed up.

In [ ]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

#specific hospital data from the PDFs
combined_context = """
[DOC 1]: Lakshmi Narayan, Age 61, Admitted 12 Aug 2024, ECG abnormal.
[DOC 2]: Neha Sharma, Age 29, Admitted 14 Aug 2024, Suspected gastritis.
[DOC 3]: Mohammed Irfan, Age 38, Admitted 16 Aug 2024, Suspected viral fever.
"""

# Fixed Query: Removed the variable name from the string
query = "Compare the ages and admission dates of all patients."

response = client.chat.completions.create(
    messages=[
        # Human-style fix: Clearly tell the AI what the context is
        {"role": "system", "content": f"Use the following patient data to answer: {combined_context}"},
        {"role": "user", "content": query}
    ],
    model="llama-3.1-8b-instant",
    temperature=0.1 # Keeps it factual to avoid hallucinations
)

print("Holistic Result:\n", response.choices[0].message.content)

Holistic Result:
 Based on the provided patient data, here's a comparison of the ages and admission dates:

1. **Lakshmi Narayan**:
   - Age: 61
   - Admission Date: 12 Aug 2024

2. **Neha Sharma**:
   - Age: 29
   - Admission Date: 14 Aug 2024

3. **Mohammed Irfan**:
   - Age: 38
   - Admission Date: 16 Aug 2024

Comparing the ages:
- Lakshmi Narayan is the oldest at 61 years.
- Mohammed Irfan is in the middle at 38 years.
- Neha Sharma is the youngest at 29 years.

Comparing the admission dates:
- Lakshmi Narayan was admitted first on 12 Aug 2024.
- Neha Sharma was admitted second on 14 Aug 2024.
- Mohammed Irfan was admitted last on 16 Aug 2024.


In [13]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# The data from your specific hospital PDFs
combined_context = """
[DOC 1]: Lakshmi Narayan, Age 61, Admitted 12 Aug 2024.
[DOC 2]: Neha Sharma, Age 29, Admitted 14 Aug 2024.
[DOC 3]: Mohammed Irfan, Age 38, Admitted 16 Aug 2024.
"""

# The Query
query = "Compare the ages and admission dates of all patients."

response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": f"You are a factual agent. Use this data: {combined_context}"},
        {"role": "user", "content": query}
    ],
    model="llama-3.1-8b-instant",
    temperature=0.5 # Forced deterministic output to prevent hallucinations
)

print("Holistic Result (Temp 0):\n", response.choices[0].message.content)

Holistic Result (Temp 0):
 Based on the provided data, here's a comparison of the ages and admission dates of all patients:

1. **Lakshmi Narayan**:
   - Age: 61
   - Admission Date: 12 Aug 2024

2. **Neha Sharma**:
   - Age: 29
   - Admission Date: 14 Aug 2024

3. **Mohammed Irfan**:
   - Age: 38
   - Admission Date: 16 Aug 2024

Comparing the ages:
- The youngest patient is **Neha Sharma** with an age of 29.
- The oldest patient is **Lakshmi Narayan** with an age of 61.
- **Mohammed Irfan** is in between with an age of 38.

Comparing the admission dates:
- The earliest admission date is **Lakshmi Narayan** on 12 Aug 2024.
- The latest admission date is **Mohammed Irfan** on 16 Aug 2024.
- **Neha Sharma** was admitted on 14 Aug 2024, which is in between the two dates.


In [14]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# The data from your specific hospital PDFs
combined_context = """
[DOC 1]: Lakshmi Narayan, Age 61, Admitted 12 Aug 2024.
[DOC 2]: Neha Sharma, Age 29, Admitted 14 Aug 2024.
[DOC 3]: Mohammed Irfan, Age 38, Admitted 16 Aug 2024.
"""

# The Query
query = "Compare the ages and admission dates of all patients."

response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": f"You are a factual agent. Use this data: {combined_context}"},
        {"role": "user", "content": query}
    ],
    model="llama-3.1-8b-instant",
    temperature=1  # Forced deterministic output to prevent hallucinations
)

print("Holistic Result (Temp 0):\n", response.choices[0].message.content)

Holistic Result (Temp 0):
 Based on the provided data, here's a comparison of the ages and admission dates of all patients:

**Age Comparison:**

1. Lakshmi Narayan - 61 years old
2. Neha Sharma - 29 years old
3. Mohammed Irfan - 38 years old

- Neha Sharma (29) is the youngest patient.
- Mohammed Irfan (38) is younger than Lakshmi Narayan (61).
- Lakshmi Narayan (61) is the oldest patient.

**Admission Dates:**

1. Lakshmi Narayan - 12 Aug 2024
2. Neha Sharma - 14 Aug 2024
3. Mohammed Irfan - 16 Aug 2024

- Lakshmi Narayan was admitted first on 12 Aug 2024.
- Neha Sharma was admitted two days later, on 14 Aug 2024.
- Mohammed Irfan was admitted two days after Neha Sharma, on 16 Aug 2024.


1. Temperature 0: The "Factual" Baseline

2. Temperature 0.5: The "Balanced" Output
Observation:noticed the AI started adding descriptive phrases like "Mohammed Irfan is in between" or "earliest admission date."

3. Temperature 1.0: The "Creative" Narrative
Observation: The AI began calculating the time between dates (e.g., "admitted two days later")

In [15]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# The data from your specific hospital PDFs
combined_context = """
[DOC 1]: Lakshmi Narayan, Age 61, Admitted 12 Aug 2024.
[DOC 2]: Neha Sharma, Age 29, Admitted 14 Aug 2024.
[DOC 3]: Mohammed Irfan, Age 38, Admitted 16 Aug 2024.
"""

def run_prompt_experiment(technique, system_msg, user_msg):
    print(f"\n--- Running {technique} ---")
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ],
        model="llama-3.1-8b-instant",
        temperature=0
    )
    print(response.choices[0].message.content)

# 1. ZERO-SHOT (Direct request, no examples)
zero_shot_sys = "You are a factual assistant. Answer strictly based on context."
zero_shot_user = f"List all patient names from this data: {combined_context}"
run_prompt_experiment("Zero-Shot", zero_shot_sys, zero_shot_user)

# 2. FEW-SHOT (Giving an example of the desired format)
few_shot_sys = "You are an extractor. Follow the user's example format exactly."
few_shot_user = f"""
Example: [NAME] | [AGE]
Data: [DOC 1]: Lakshmi Narayan, Age 61.
Output: LAKSHMI NARAYAN | 61

Now do the same for all patients in this data: {combined_context}
"""
run_prompt_experiment("Few-Shot", few_shot_sys, few_shot_user)

# 3. CHAIN-OF-THOUGHT (Thinking step-by-step to prevent logic errors)
cot_sys = "You are a logical analyst. Think step-by-step before answering."
cot_user = f"""
Data: {combined_context}
Question: Who is the oldest patient and how many years older are they than the youngest?
"""
run_prompt_experiment("Chain-of-Thought", cot_sys, cot_user)


--- Running Zero-Shot ---
Here's the list of patient names:

1. Lakshmi Narayan
2. Neha Sharma
3. Mohammed Irfan

--- Running Few-Shot ---
[DOC 1]: LAKSHMI NARAYAN | 61
[DOC 2]: NEHA SHARMA | 29
[DOC 3]: MOHAMMED IRFAN | 38

--- Running Chain-of-Thought ---
To determine the oldest patient and the age difference between the oldest and the youngest, I will analyze the data step by step:

1. Identify the ages of all patients:
   - Lakshmi Narayan: 61 years
   - Neha Sharma: 29 years
   - Mohammed Irfan: 38 years

2. Determine the oldest patient:
   - The oldest patient is Lakshmi Narayan, who is 61 years old.

3. Identify the youngest patient:
   - The youngest patient is Neha Sharma, who is 29 years old.

4. Calculate the age difference between the oldest and the youngest:
   - Age difference = Age of oldest patient - Age of youngest patient
   - Age difference = 61 - 29
   - Age difference = 32 years

Therefore, the oldest patient is Lakshmi Narayan, and they are 32 years older than th

Developer Note: Prompt Strategy Findings

Zero-Shot: Used for general QA; it provides direct answers but can sometimes be wordy.

Few-Shot: Best for Extraction; providing a single example format (e.g., Name | Age) ensures the agent outputs data in a clean, predictable structure .

Chain-of-Thought (CoT): Essential for Holistic Insights; forcing the model to "think step-by-step" ensures accurate comparisons between patients (like age gaps) and effectively rectifies hallucinations